<a href="https://colab.research.google.com/github/lilimelgar/systematic_queries_with_pandas_and_regex/blob/main/BeyondCtrlF_SystematicQueriesWithPandas_Liliana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Querying-CERA-letters-with-regular-expressions-in-Python" data-toc-modified-id="Querying-CERA-letters-with-regular-expressions-in-Python-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Querying CERA letters with regular expressions in Python</a></span></li><li><span><a href="#Loading-file-with-letters'-metadata-+-content" data-toc-modified-id="Loading-file-with-letters'-metadata-+-content-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading file with letters' metadata + content</a></span></li><li><span><a href="#Querying-the-entire-set-with-Regex" data-toc-modified-id="Querying-the-entire-set-with-Regex-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Querying the entire set with Regex</a></span></li></ul></div>

# Querying CERA letters with regular expressions in Python

Notebook prepared by Liliana Melgar for SKILLNET project

Last update: March 31, 2021 (includes a for loop to create a dataframe that includes all the queries (patterns) aggregated into one dataframe
Created: March 16, 2021

Sources:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html

This notebook shows a basic way to use Python regular expressions for querying data, in this case different volumes of the CERA letters. The data from the editions was converted to a table which includes the basic page metadata and the content of the letters downloaded from the CERA website.

First batch:
- http://mateo.uni-mannheim.de/cera/autoren/schurzfleisch_cera.html

Second batch:
- http://mateo.uni-mannheim.de/cera/autoren/hessus_cera.html
- http://mateo.uni-mannheim.de/cera/autoren/reinesius_cera.html
- http://mateo.uni-mannheim.de/cera/autoren/richter_cera.html

In [ ]:
import matplotlib
import pandas as pd
import re
from IPython.display import clear_output
import numpy as np

# to export to Excel
import openpyxl

# from IPython.display import display, HTML
# # display(HTML("<style>.container { width:95% !important; }</style>"))


# Loading file with letters' metadata + content

In [ ]:
# This is the code to indicate to this notebook that the file will be loaded from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the file as a pandas (pd) data frame, in this case a .csv file which contains tabular data

# second batch = Hessus, Helius; Reinesius, Thomas; Richter, Georg
# file = pd.read_csv("CERA-Batch2-hessus-reinesius-richter-ALL_v20210330_DownloadedFromOR-Clean.csv", sep = "ℑ", engine='python')
# remember to upload file to "sample_data"

path = "/content/sample_data/CERA-Batch2-hessus-reinesius-richter-ALL_v20210330_DownloadedFromOR-Clean.csv"
file = pd.read_csv(path, sep = "ℑ", engine='python')


In [ ]:
# Get familiar with the columns and data types of the data

file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3156 entries, 0 to 3155
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   pageId   3156 non-null   object
 1   edition  3156 non-null   object
 2   page     3156 non-null   object
 3   link     3156 non-null   object
 4   text     3156 non-null   object
dtypes: object(5)
memory usage: 123.4+ KB


In [ ]:
# Change the type of data of the "Content" column to string

file['text'] = file['text'].astype('string')

file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3156 entries, 0 to 3155
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   pageId   3156 non-null   object
 1   edition  3156 non-null   object
 2   page     3156 non-null   object
 3   link     3156 non-null   object
 4   text     3156 non-null   string
dtypes: object(4), string(1)
memory usage: 123.4+ KB


In [ ]:
# store the copy to work on from now

letters = file.copy()

In [ ]:
letters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3156 entries, 0 to 3155
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   pageId   3156 non-null   object
 1   edition  3156 non-null   object
 2   page     3156 non-null   object
 3   link     3156 non-null   object
 4   text     3156 non-null   string
dtypes: object(4), string(1)
memory usage: 123.4+ KB


In [ ]:
letters.head()

,pageId,edition,page,link,text
0,hessus_01_s001,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s001,http://mateo.uni-mannheim.de/cera/camerarius1/...,"NARRATIO DE H. EOBANO HESSO, COMPREHENdens men..."
1,hessus_01_s002,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s002,http://mateo.uni-mannheim.de/cera/camerarius1/...,SIGNVM EOBANICVM CVM ELOGIO IPSIVS. /[gap: ill...
2,hessus_01_s003,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s003,http://mateo.uni-mannheim.de/cera/camerarius1/...,DE H. EOBANO HESSO NARRATIO IOACHIMI CAMErarij...
3,hessus_01_s004,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s004,http://mateo.uni-mannheim.de/cera/camerarius1/...,"in acerrima aegritudine, neque in tanta hominu..."
4,hessus_01_s005,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s005,http://mateo.uni-mannheim.de/cera/camerarius1/...,"secundum Poetam, floribus austri sane procello..."


# Querying the entire set with Regex

In [ ]:
# findall(): Return a list containing every occurrence, but in this case using the "text" column of the entire 
# dataframe 
# (notice the difference in the syntax between the way to use Regex in a dataframe (this cell) and in text only 
# (no dataframe: cell above))

# pattern to query for all variations of "republica literaria" listed here: https://surfdrive.surf.nl/files/index.php/s/PZ0H5KAILzbrvpW
pattern1a = re.compile('re.*p.* lit+erar.*|lit+erar.* re.*p.*', re.IGNORECASE) #with "p", case insensitive)
pattern1b = re.compile('re.* lit+erar.*|lit+erar.* re.*', re.IGNORECASE) #without "p", case insensitive

# pattern to query for all variations of republica academica (with or without words in between, e.g., republicae gerendis academica)
pattern2 = re.compile('re\w*?p.*\s\w*?\sacad.*', re.IGNORECASE)

# pattern to query for all variations of commercio literario
pattern3 = re.compile('commerc.*lit+erar.*|lit+erar.*commerc.*', re.IGNORECASE)

# pattern to query for variations of academia
pattern4 = re.compile('\s?academi.*', re.IGNORECASE)

# pattern to query for variations of communium amicorum
pattern5 = re.compile('commun.*\samic.*', re.IGNORECASE)

# pattern to query for variations of communium studiorum
pattern6 = re.compile('commun.*\sstu.*', re.IGNORECASE)

# pattern to query for variations of forum lit('t)erarium
pattern7 = re.compile('for.*\slit+erar.*', re.IGNORECASE)

# pattern to query for variations of lit('t)erarum officium
pattern8 = re.compile('lit+erar.*\soffic.*|offic.*\slit+erar.*', re.IGNORECASE)

# pattern to query for variations of literarum commodo
pattern9 = re.compile('lit+erar.*\scommo.*', re.IGNORECASE)

# pattern to query for variations of \literarum humaniorum\
pattern10 = re.compile('lit+erar.*\shuman.*', re.IGNORECASE)

# pattern to query for variations of \nationis lit('t)erariorum\
pattern11 = re.compile('natio.*\slit+erar.*', re.IGNORECASE)

# pattern to query for variations of \Orbi lit('t)erato\
pattern12 = re.compile('lit+erato\sorbi|orbi\slit+erato', re.IGNORECASE)

# pattern to query for variations of \universum studiosorum\
pattern13 = re.compile('univers.*\sstudi.*', re.IGNORECASE)

# pattern to query for variations of \republica\ alone
pattern14 = re.compile(r'\bre\w?\s?public.*\b', re.IGNORECASE)

# pattern to query for variations of \literaria\ alone
pattern15 = re.compile(r'\blit+era.*\b', re.IGNORECASE)


#other interesting queries:
# stud.*nostra
# academia nostra\n


In [ ]:
# create a dictionary with patterns

patternName = ['pat01a', 'pat01b', 'pat02', 'pat03', 'pat04', 'pat05', 'pat06', 'pat07', 'pat08', 'pat09', 'pat10', 'pat11', 'pat12', 'pat13', 'pat14', 'pat15']

pattern = [pattern1a, pattern1b, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7, pattern8, pattern9, pattern10, pattern11, pattern12, pattern13, pattern14, pattern15]

patterns_dictionary = dict(zip(patternName, pattern))

In [ ]:
patterns_dictionary

{'pat01a': re.compile(r're.*p.* lit+erar.*|lit+erar.* re.*p.*',
 re.IGNORECASE|re.UNICODE),
 'pat01b': re.compile(r're.* lit+erar.*|lit+erar.* re.*',
 re.IGNORECASE|re.UNICODE),
 'pat02': re.compile(r're\w*?p.*\s\w*?\sacad.*', re.IGNORECASE|re.UNICODE),
 'pat03': re.compile(r'commerc.*lit+erar.*|lit+erar.*commerc.*',
 re.IGNORECASE|re.UNICODE),
 'pat04': re.compile(r'\s?academi.*', re.IGNORECASE|re.UNICODE),
 'pat05': re.compile(r'commun.*\samic.*', re.IGNORECASE|re.UNICODE),
 'pat06': re.compile(r'commun.*\sstu.*', re.IGNORECASE|re.UNICODE),
 'pat07': re.compile(r'for.*\slit+erar.*', re.IGNORECASE|re.UNICODE),
 'pat08': re.compile(r'lit+erar.*\soffic.*|offic.*\slit+erar.*',
 re.IGNORECASE|re.UNICODE),
 'pat09': re.compile(r'lit+erar.*\scommo.*', re.IGNORECASE|re.UNICODE),
 'pat10': re.compile(r'lit+erar.*\shuman.*', re.IGNORECASE|re.UNICODE),
 'pat11': re.compile(r'natio.*\slit+erar.*', re.IGNORECASE|re.UNICODE),
 'pat12': re.compile(r'lit+erato\sorbi|orbi\slit+erato',
 re.IGNORECASE|

In [ ]:
# # get the list of keys and values from the dictionary (to use it later in creating the column with the pattern name)

key_list = list(patterns_dictionary.keys())
value_list = list(patterns_dictionary.values())

In [ ]:
# iterating over the dictionary of patterns 
# and using each value for the query

for pattern in patterns_dictionary.values():
    query = letters.text.str.findall(pattern)
    # create a copy of original letters dataframe
    queryOutput = letters.copy()
    # add a column with matching terms which contains the output of the query
    queryOutput['matchingTerms'] = query
    # apply function to convert the list of matching terms to strings separated by '|'
    queryOutput['matchingTerms'] = queryOutput.apply(lambda x: ('|'.join([str(s) for s in x['matchingTerms']])), axis = 1)
    # replace empty space left in the cell with nan
    queryOutput['matchingTerms'] = queryOutput['matchingTerms'].str.strip().replace('',np.nan)
    # obtain the name of the pattern (key) from the dictionary using the value (pattern)
    queryOutput['patternName'] = (key_list[value_list.index(pattern)])
    # make a copy of the dataframe
    queryResults = queryOutput.copy()
    # create a dataframe with the query results only (drop empty rows)
    queryResultsPerPage = queryResults.dropna(how='any', subset=['matchingTerms'])


In [ ]:
queryResultsPerPage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053 entries, 0 to 3153
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   pageId         1053 non-null   object
 1   edition        1053 non-null   object
 2   page           1053 non-null   object
 3   link           1053 non-null   object
 4   text           1053 non-null   string
 5   matchingTerms  1053 non-null   object
 6   patternName    1053 non-null   object
dtypes: object(6), string(1)
memory usage: 65.8+ KB


In [ ]:
queryResultsPerPage.head()

,pageId,edition,page,link,text,matchingTerms,patternName
0,hessus_01_s001,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s001,http://mateo.uni-mannheim.de/cera/camerarius1/...,"NARRATIO DE H. EOBANO HESSO, COMPREHENdens men...",literatae: Cum quibusdam Camerarij et,pat15
4,hessus_01_s005,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s005,http://mateo.uni-mannheim.de/cera/camerarius1/...,"secundum Poetam, floribus austri sane procello...","literas, sine quibus persuasum",pat15
7,hessus_01_s008,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s008,http://mateo.uni-mannheim.de/cera/camerarius1/...,"incorrecta et mendosa diuulgata sunt, et te no...",literas solerct,pat15
10,hessus_01_s011,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s011,http://mateo.uni-mannheim.de/cera/camerarius1/...,"abhorret a uero, in pago natum esse, cui est n...","litterarum. Verisimile est, quod|litterarum st...",pat15
11,hessus_01_s012,Hessus_01_Camerarius-Narratio-EobanoHesso,image: s012,http://mateo.uni-mannheim.de/cera/camerarius1/...,"studeret, ut pueri inprimis Grammaticas praece...",literarum tum celcbris,pat15


In [ ]:
# store output file
# queryResultsPerPage.to_excel('QueryOutput_batch2_Hessus-Reinesius-Richter.xlsx', sheet_name='queryResultsBatch2')
queryResultsPerPage.to_excel("/content/sample_data/queryResultsBatch2.xlsx", sheet_name="queryResultsBatch2")